<a href="https://colab.research.google.com/github/nishres/streamflow_ml/blob/main/11_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install xgboost
!pip install hydroeval

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import hydroeval as he
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc
from xgboost import plot_importance
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from google.colab import output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def get_train_val_test2(basin_name):
    available_data = {
        'Babai_Chepang':[4653,6669,3652,4652,6670,7669],
        'Bagmati_Padheradovan':[3652,7000,7001,8945,8949,9731],
        #'Balephi_Jalbire':[3652,7100,7101,8745,8746,9495],
        'Bhotekoshi_Barabise':[5310,6633,7670,8400,6726,6939],
        'Budhigandaki_Arughat':[5654,8714,3652,4657,4658,5647],
        'Budhiganga_Chitra':[3823,5650,5651,6251,6252,6939],
        'Chamelia_Nayalbadi':[3652,7200,7201,8745,8746,9495],
        'Dudhkoshi_Rabuwabazar':[3652,5870,5871,7059,7305,9130],
        # 'Indrawati_Dolalghat':[5844,7029,7670,8270,8271,8765],
        'Kaligandaki_Anshing':[3652,7200,7201,8745,8746,9495],
        #'Likhu_Sangutar':[6193,7198,5479,6102,4748,5112],
        'Madi_Shisaghat':[3755,7300,7301,8800,8801,9753],
        'Marikhola_Nayagaon':[3652,7200,7201,8800,8801,9814],
        'Marsyangdi_Bhakundebesi':[3683,4883,5326,6208,4884,5316],
        #'Modikhola_Nayapul':[5338,8795,3652,5329,8796,9495],
        'Myagdikhola_Mangalghat':[6991,9495,3712,4887,6018,6498],
        'Rapti_Bagasotigaon':[3652,6600,6601,8045,8046,8936],
        'Rapti_Jalkundi':[3652,7300,7301,8800,8801,9861],
        # 'Rapti_Kusum':[4748,7600,7601,9145,9146,10122],
        # 'Sabhyakhola_Tumlingtar':[7670,9495,6209,7624,5098,6192],
        # 'Saradakhola_Daradhunga':[3652,6939,7670,8970,8971,9861],
        'Seti_Bangga':[7213,9861,3987,5600,5601,6604],
        'Seti_Gopaghat':[3652,7200,7201,8745,8746,9131],
        'Setigandaki_Damauli':[3652,7300,7301,8800,8801,9861],
        #'Sinjhakhola_Diware':[3652,7200,7201,8745,8746,9495],
        'Sunkoshi_Khurkot':[5113,6458,8766,9495,6459,7258],
        'Sunkoshi_Pachwarghat':[3652,7200,7201,8751,8933,9495],
        #'Tadi_Belkot':[4337,7500,7501,9495,3652,4309],
        'Tamakoshi_Busti':[5113,6939,7670,8400,8766,9495],
        'Tamor_Majhitar':[3652,6132,6677,8400,6209,6644],
        'Tamor_Mulghat':[4868,7400,7401,8765,4183,4747],
        'Thulibheri_Rimna':[3652,6600,6601,8533,8534,9861],
        'Tilanadi_Nagma':[3652,6400,8035,9495,6401,7945],
        'Trishuli_Betrawati':[3652,7200,7201,8745,8746,9495],

    }
    for key in available_data.keys():
        if key in basin_name:
            train_s = available_data[key][0]
            train_e = available_data[key][1]
            val_s = available_data[key][2]
            val_e = available_data[key][3]
            test_s = available_data[key][4]
            test_e = available_data[key][5]
    #print(train_s,train_e,val_s, val_e, test_s, test_e)
    return train_s, train_e,val_s, val_e, test_s, test_e

In [ ]:
in_path = '/content/drive/MyDrive/2_2/00_Raw'

all_files = os.listdir(in_path)
for i in range(len(all_files)):
  print(i,all_files[i])


0 Babai_Chepang_basin_avg.csv
1 Budhiganga_Chitra_basin_avg.csv
2 Chamelia_Nayalbadi_basin_avg.csv
3 Dudhkoshi_Rabuwabazar_basin_avg.csv
4 Marikhola_Nayagaon_basin_avg.csv
5 Kaligandaki_Anshing_basin_avg.csv
6 Myagdikhola_Mangalghat_basin_avg.csv
7 Rapti_Jalkundi_basin_avg.csv
8 Rapti_Bagasotigaon_basin_avg.csv
9 Seti_Bangga_basin_avg.csv
10 Seti_Gopaghat_basin_avg.csv
11 Setigandaki_Damauli_basin_avg.csv
12 Sunkoshi_Khurkot_basin_avg.csv
13 Sunkoshi_Pachwarghat_basin_avg.csv
14 Tamakoshi_Busti_basin_avg.csv
15 Tamor_Mulghat_basin_avg.csv
16 Tamor_Majhitar_basin_avg.csv
17 Thulibheri_Rimna_basin_avg.csv
18 Tilanadi_Nagma_basin_avg.csv
19 Trishuli_Betrawati_basin_avg.csv
20 Budhigandaki_Arughat_basin_avg.csv


In [ ]:
def get_data(b_name,r_df):
    r_df = r_df[['t2m_mean','src_mean','snowc_mean','tp_sum','ssro_sum', 'e_sum' ,'sf_sum','Observed']]
    train_start, train_end,val_start, val_end, test_start, test_end = get_train_val_test2(b_name)
    #print(train_start, train_end, val_start, val_end, test_start, test_end)
    df_tr = r_df[train_start:train_end]
    df_vl = r_df[val_start:val_end]
    df_ts = r_df[test_start:test_end]
    return normalize(df_tr),  normalize(df_vl), normalize(df_ts)  #

In [ ]:
def normalize(udf):
    #scaler = MinMaxScaler()
    scaler = StandardScaler()
    udf = udf[[c for c in udf if c not in ['time']]]
    #print(udf)
    column_transformer = ColumnTransformer(
        transformers=[
            ('scaler', scaler, ['t2m_mean', 'src_mean', 'snowc_mean', 'tp_sum','ssro_sum','e_sum' ,'sf_sum'])
        ],
        remainder='passthrough'
    )

    transformed_data = column_transformer.fit_transform(udf)
    df_normalized = pd.DataFrame(transformed_data, columns=udf.columns)
    del udf
    #print()
    #print(df_normalized)
    return df_normalized

In [ ]:
n_days = [7,10]

In [ ]:
for i in range(len(all_files)):
    basin_name = all_files[i][:-14]
    raw_df = pd.read_csv(os.path.join(in_path,all_files[i]))
    raw_df.iloc[:-1,18] = raw_df.iloc[1:,18]
    df_train, df_val, df_test = get_data(all_files[i],raw_df)
    #train_start, train_end, val_start, val_end, test_start, test_end = get_train_val_test2(all_files[i])

    out_path = '/content/drive/MyDrive/16_IoE_GC/04_XGB_longer/' + basin_name
    if not os.path.exists(out_path):
      os.makedirs(out_path)

    out_model_path = os.path.join(out_path, 'Model')
    out_plot_path = os.path.join(out_path, 'Plot')
    out_Q_path = os.path.join(out_path, 'Predicted')
    if not os.path.exists(out_model_path):
      os.makedirs(out_model_path)
    if not os.path.exists(out_plot_path):
      os.makedirs(out_plot_path)
    if not os.path.exists(out_Q_path):
      os.makedirs(out_Q_path)
    df_summary = pd.DataFrame(columns=['Basin','Days','NSE_tr','NSE_vl','NSE_ts','KGE_tr','KGE_vl','KGE_ts','r_tr','r_vl','r_ts','alpha_tr','alpha_vl','alpha_ts','beta_tr','beta_vl','beta_ts' ])
    loc = 0

    for day in n_days:
        output.clear()
        suffix =  '_D' + str(f'{day:01d}') #'_d' + str(f'{dropout:.3f}') +
        out_model = os.path.join(out_model_path, basin_name + suffix + '.json')
        print(out_model)

        ######################
        x_train, y_train = df_train.drop('Observed', axis=1), df_train[['Observed']]
        x_val, y_val = df_val.drop('Observed', axis=1), df_val[['Observed']]
        x_test, y_test = df_val.drop('Observed', axis=1), df_val[['Observed']]

        #del df_train, df_val, df_test

        if day > 0:
            x_train, y_train = x_train[:-day], y_train[day:]
            x_val, y_val = x_val[:-day], y_val[day:]
            x_test, y_test = x_test[:-day], y_test[day:]
        else:
              x_train, y_train = x_train, y_train
              x_val, y_val = x_val, y_val
              x_test, y_test = x_test, y_test

        dtrain_reg = xgb.DMatrix(x_train, y_train, enable_categorical=True)
        dval_reg = xgb.DMatrix(x_val, y_val, enable_categorical=True)
        dtest_reg = xgb.DMatrix(x_test, y_test, enable_categorical=True)

        params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse'
        }
        evals = [(dtrain_reg, "train"), (dval_reg, "validation")]
        n = 100
        model = xgb.train(
          params=params,
          dtrain=dtrain_reg,
          num_boost_round=n,
          evals=evals,
          verbose_eval=True,
          early_stopping_rounds=10
        )

        #model.save_model(out_model)
        del out_model

        y_trpred = model.predict(dtrain_reg)
        y_trpred = pd.DataFrame(y_trpred)
        y_trpred = y_trpred.set_index(y_train.index[:len(y_trpred)])
        result_tr = y_train.join(y_trpred)
        result_tr.columns = ['actual', 'predicted']
        result_tr.to_csv(os.path.join(out_Q_path, basin_name + suffix + '_Train.csv'))

        simulations_tr = result_tr['predicted']
        actuals_tr = result_tr['actual']
        del result_tr, y_trpred, dtrain_reg, x_train, y_train

        nse_tr = he.evaluator(he.nse, simulations_tr, actuals_tr)
        kge_tr, r_tr, alpha_tr, beta_tr = he.evaluator(he.kge, simulations_tr, actuals_tr)

        y_vpred = model.predict(dval_reg)
        y_vpred = pd.DataFrame(y_vpred)
        y_vpred = y_vpred.set_index(y_val.index[:len(y_vpred)])
        result_vl = y_val.join(y_vpred)
        result_vl.columns = ['actual', 'predicted']
        result_vl.to_csv(os.path.join(out_Q_path, basin_name + suffix + '_Val.csv'))

        simulations_vl = result_vl['predicted']
        actuals_vl = result_vl['actual']
        del result_vl, y_vpred, dval_reg, x_val, y_val

        nse_vl = he.evaluator(he.nse, simulations_vl, actuals_vl)
        kge_vl, r_vl, alpha_vl, beta_vl = he.evaluator(he.kge, simulations_vl, actuals_vl)

        y_tspred = model.predict(dtest_reg)
        y_tspred = pd.DataFrame(y_tspred)
        y_tspred = y_tspred.set_index(y_test.index[:len(y_tspred)])
        result_ts = y_test.join(y_tspred)
        result_ts.columns = ['actual', 'predicted']
        result_ts.to_csv(os.path.join(out_Q_path, basin_name + suffix + '_Test.csv'))

        simulations_ts = result_ts['predicted']
        actuals_ts = result_ts['actual']
        del result_ts, y_tspred, dtest_reg, x_test, y_test

        nse_ts = he.evaluator(he.nse, simulations_ts, actuals_ts)
        kge_ts, r_ts, alpha_ts, beta_ts = he.evaluator(he.kge, simulations_ts, actuals_ts)

        df_summary.loc[loc] = [basin_name, day, nse_tr[0], nse_vl[0], nse_ts[0], kge_tr[0], kge_vl[0], kge_ts[0],
                                      r_tr[0], r_vl[0], r_ts[0], alpha_tr[0], alpha_vl[0], alpha_ts[0], beta_tr[0], beta_vl[0], beta_ts[0] ]
        loc += 1

        # #Hydrograph Plots
        # fig, axes = plt.subplots(1,1, figsize=(15,8))
        # fig.suptitle(f'Training Set {basin_name}_{suffix}')
        # axes.plot(simulations_tr[:1500],label = 'Predicted')
        # axes.plot(actuals_tr[:1500],label = 'Actual')
        # plt.legend()
        # plt.show()
        # out_plot = os.path.join(out_plot_path, basin_name + '_Train' + suffix + '.png' )
        # fig.savefig(out_plot)

        # fig, axes = plt.subplots(1,1, figsize=(15,8))
        # fig.suptitle(f'Validation Set {basin_name}_{suffix}')
        # axes.plot(simulations_vl[:760],label = 'Predicted')
        # axes.plot(actuals_vl[:760],label = 'Actual')
        # plt.legend()
        # plt.show()
        # out_plot = os.path.join(out_plot_path, basin_name + '_Val' + suffix + '.png' )
        # fig.savefig(out_plot)

        # fig, axes = plt.subplots(1,1, figsize=(15,8))
        # fig.suptitle(f'Testing Set {basin_name}_{suffix}')
        # axes.plot(simulations_ts[:760],label = 'Predicted')
        # axes.plot(actuals_ts[:760],label = 'Actual')
        # plt.legend()
        # plt.show()
        # out_plot = os.path.join(out_plot_path, basin_name + '_Test' + suffix + '.png' )
        # fig.savefig(out_plot)
        # print()

        # #Observed vs Predicted
        # fig, axes = plt.subplots(1,3, figsize=(30, 10))
        # fig.suptitle(f'Observed vs Predicted flow at {basin_name}_{suffix}')
        # axes[0].plot(simulations_tr, actuals_tr, "o", alpha=0.4)
        # if simulations_tr.max() > actuals_tr.max():
        #   max = simulations_tr.max()
        # else:
        #   max = actuals_tr.max()
        # axes[0].plot([0,max], [0,max], "k--", alpha=0.75)
        # axes[0].set_xlabel("Predicted")
        # axes[0].set_ylabel("Observed")
        # axes[0].set_title('Train')

        # axes[1].plot(simulations_vl, actuals_vl, "o", alpha=0.4)
        # if simulations_vl.max() > actuals_vl.max():
        #   max = simulations_vl.max()
        # else:
        #   max = actuals_vl.max()
        # axes[1].plot([0,max], [0,max], "k--", alpha=0.75)
        # axes[1].set_xlabel("Predicted")
        # axes[1].set_ylabel("Observed")
        # axes[1].set_title('Validation')

        # axes[2].plot(simulations_ts, actuals_ts, "o", alpha=0.4)
        # if simulations_ts.max() > actuals_ts.max():
        #   max = simulations_ts.max()
        # else:
        #   max = actuals_ts.max()
        # axes[2].plot([0,max], [0,max], "k--", alpha=0.75)
        # axes[2].set_xlabel("Predicted")
        # axes[2].set_ylabel("Observed")
        # axes[2].set_title('Test')
        # plt.legend()
        # plt.show()
        # out_plot = os.path.join(out_plot_path, basin_name + '_OvP' + suffix + '.png' )
        # fig.savefig(out_plot)

        # ###############
        # #FDC
        # sort_sim_tr = np.sort(simulations_tr)[::-1]
        # sort_act_tr = np.sort(actuals_tr)[::-1]
        # sort_sim_vl = np.sort(simulations_vl)[::-1]
        # sort_act_vl = np.sort(actuals_vl)[::-1]
        # sort_sim_ts = np.sort(simulations_ts)[::-1]
        # sort_act_ts = np.sort(actuals_ts)[::-1]
        # exceedence_tr = np.arange(1.,len(sort_sim_tr)+1) / len(sort_sim_tr)
        # exceedence_vl = np.arange(1.,len(sort_sim_vl)+1) / len(sort_sim_vl)
        # exceedence_ts = np.arange(1.,len(sort_sim_ts)+1) / len(sort_sim_ts)

        # fig, axes = plt.subplots(1,3, figsize=(19,5))
        # fig.suptitle(f'Flow Duration Curve for {basin_name}_{suffix}')
        # axes[0].plot(100*exceedence_tr,sort_sim_tr,label = 'Predicted')
        # axes[0].plot(100*exceedence_tr,sort_act_tr,label = 'Actual')
        # axes[0].set_xlabel("Exceedence [%]")
        # axes[0].set_ylabel("Min-Max Normalized Flow")
        # axes[0].set_title('Train',y = -.20)
        # axes[1].plot(100*exceedence_vl,sort_sim_vl,label = 'Predicted')
        # axes[1].plot(100*exceedence_vl,sort_act_vl,label = 'Actual')
        # axes[1].set_xlabel("Exceedence [%]")
        # axes[1].set_ylabel("Min-Max Normalized Flow")
        # axes[1].set_title('Validation',y = -.20)
        # axes[2].plot(100*exceedence_ts,sort_sim_ts,label = 'Predicted')
        # axes[2].plot(100*exceedence_ts,sort_act_ts,label = 'Actual')
        # axes[2].set_xlabel("Exceedence [%]")
        # axes[2].set_ylabel("Min-Max Normalized Flow")
        # axes[2].set_title('Test',y = -.20)
        # plt.legend()
        # plt.show()
        # out_plot = os.path.join(out_plot_path, basin_name + '_FDC' + suffix + '.png' )
        # fig.savefig(out_plot)

        # del fig, axes, sort_sim_tr, sort_act_tr, sort_sim_vl, sort_act_vl, sort_sim_ts, sort_act_ts, exceedence_tr, exceedence_vl, exceedence_ts

    df_summary.to_csv(os.path.join(out_path, basin_name + 'Summary.csv'),index = False)
    del df_summary
    gc.collect()


/content/drive/MyDrive/16_IoE_GC/04_XGB_longer/Budhigandaki_Arughat/Model/Budhigandaki_Arughat_D10.json
[0]	train-rmse:120.56951	validation-rmse:165.27839
[1]	train-rmse:92.43989	validation-rmse:142.03308
[2]	train-rmse:73.91463	validation-rmse:127.00470
[3]	train-rmse:61.58538	validation-rmse:116.20380
[4]	train-rmse:53.75130	validation-rmse:108.55292
[5]	train-rmse:48.86267	validation-rmse:103.54400
[6]	train-rmse:45.64726	validation-rmse:101.46267
[7]	train-rmse:43.30419	validation-rmse:101.70306
[8]	train-rmse:41.98512	validation-rmse:100.54163
[9]	train-rmse:40.97913	validation-rmse:99.97505
[10]	train-rmse:40.21044	validation-rmse:99.42748
[11]	train-rmse:39.02387	validation-rmse:100.02799
[12]	train-rmse:37.58040	validation-rmse:100.04997
[13]	train-rmse:36.15560	validation-rmse:100.17068
[14]	train-rmse:35.47895	validation-rmse:100.41862
[15]	train-rmse:34.72992	validation-rmse:100.12401
[16]	train-rmse:34.45472	validation-rmse:100.88952
[17]	train-rmse:34.07637	validation-rmse

In [ ]:
df_train

,t2m_mean,src_mean,snowc_mean,tp_sum,ssro_sum,e_sum,sf_sum,Observed
0,-1.630284,-0.934604,-0.161401,-0.492981,-0.540386,0.908289,-0.147014,19.30
1,-1.485027,-0.973726,-0.162470,-0.497092,-0.543559,0.898495,-0.147014,19.00
2,-1.498182,-0.981174,-0.163130,-0.497777,-0.546761,0.855420,-0.147014,19.30
3,-1.595616,-0.966434,-0.163438,-0.500556,-0.549890,0.993760,-0.147014,18.90
4,-1.620456,-0.964375,-0.163487,-0.500802,-0.552972,1.045310,-0.147014,19.30
...,...,...,...,...,...,...,...,...
3343,-0.462112,-0.961806,-0.162954,-0.464214,-0.670869,0.778665,-0.147014,9.65
3344,-0.502245,-1.028831,-0.163429,-0.488449,-0.671932,0.674426,-0.147014,9.65
3345,-0.599865,-1.032646,-0.163487,-0.500433,-0.672994,0.700884,-0.147014,9.65
3346,-0.835616,-1.025367,-0.163487,-0.499252,-0.674071,0.686239,-0.147014,9.65


In [ ]:
df2 = df_train[:-1]

In [ ]:
df2

,t2m_mean,src_mean,snowc_mean,tp_sum,ssro_sum,e_sum,sf_sum,Observed
0,-1.630284,-0.934604,-0.161401,-0.492981,-0.540386,0.908289,-0.147014,19.30
1,-1.485027,-0.973726,-0.162470,-0.497092,-0.543559,0.898495,-0.147014,19.00
2,-1.498182,-0.981174,-0.163130,-0.497777,-0.546761,0.855420,-0.147014,19.30
3,-1.595616,-0.966434,-0.163438,-0.500556,-0.549890,0.993760,-0.147014,18.90
4,-1.620456,-0.964375,-0.163487,-0.500802,-0.552972,1.045310,-0.147014,19.30
...,...,...,...,...,...,...,...,...
3342,-0.543518,-0.875966,-0.163428,-0.464221,-0.669768,0.760229,-0.147014,10.20
3343,-0.462112,-0.961806,-0.162954,-0.464214,-0.670869,0.778665,-0.147014,9.65
3344,-0.502245,-1.028831,-0.163429,-0.488449,-0.671932,0.674426,-0.147014,9.65
3345,-0.599865,-1.032646,-0.163487,-0.500433,-0.672994,0.700884,-0.147014,9.65


In [ ]:
# Calculate feature importances
importances = model.get_score(importance_type='weight')
plot_importance(model)
plt.show()